<a href="https://colab.research.google.com/github/Ela0-o/ML_kurs2022/blob/main/domashka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Условие задачи**
На основании данных о пассажирах, которые воспользовались метро дважды за сутки, при
наличии информации о первом заходе в метро, необходимо предсказать, на какой станции
и через какой промежуток времени, этот пассажир воспользуется метро повторно

# Описание столбцов для train и test:
* id - уникальный идентификатор столбца;
* ticket_id - уникальный идентификатор билета, считается, что у одного
билета один владелец
* ticket_type_nm - тип билета
* entrance_id - уникальный id входа в станцию
* entrance_nm - название
* station_id - уникальное id станции захода
* station_nm - наименование станции захода
* line_id - уникальный id ветки на, которой находится станция
* line_nm - наименование ветки, на которой находится станция
* pass_dttm - дата валидации
* time_to_under **(столбец для предсказания)** - сколько времени прошло
между первой и второй валидацией
* label **(столбец для предсказания)** - id второй станции, на которой
произошла валидация 

In [14]:
from google.colab import drive
drive.mount('/content/gdrive')

import pandas as pd
import numpy as np

df = pd.read_csv('/content/gdrive/My Drive/train_dataset_train.csv') # Чтение файла
df


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,id,ticket_id,ticket_type_nm,entrance_id,entrance_nm,station_id,station_nm,line_id,line_nm,pass_dttm,time_to_under,label
0,1,40BD89EC85646EFB69E283F39C298E60,Пропуск FacePay,2402,Лефортово БКЛ,11007,Лефортово,11,Большая кольцевая,2022-09-12 05:00:13,216.316667,8001
1,2,126727A96489CC976A8C08E5CEB00542,СК учащегося 30 дней,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:00:54,648.183333,9011
2,3,D28CE6A9E0E5B6D213470A97CFF32485,БСК дружинника г.Москвы,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:00:55,865.333333,7022
3,4,015DA44B523C062B5BFEFF3FB0E64B9E,30 дней,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:01:13,1048.233333,2022
4,5,95B19C6F3A504727AC3EA56EB7E3E80F,КОШЕЛЕК,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:02:55,965.600000,2017
...,...,...,...,...,...,...,...,...,...,...,...,...
1091016,1091017,6A738232F8E43DFCD4027FDC0EA6CC03,Пропуск FacePay,2283,Нижегородская (второй),15005,Нижегородская,15,Некрасовская,2022-09-19 02:54:50,124.750000,15005
1091017,1091018,141174B366F14A8703903813A49374DB,Пропуск FacePay,278,Калужская (Северный),6007,Калужская,6,Калужско-Рижская,2022-09-19 03:06:02,56.066667,6007
1091018,1091019,E38493B30A7C55170806F187AF5EEA15,Пропуск FacePay,118,Каширская (Северный ),2010,Каширская,2,Замоскворецкая,2022-09-19 03:17:00,91.283333,2010
1091019,1091020,3DF2B6366631ECA06274DD2CB5B0DF24,Пропуск FacePay,2452,Воронцовская (Первый),11014,Воронцовская,11,Большая кольцевая,2022-09-19 03:23:30,47.900000,11014


# Предобработка

In [ ]:
# можно было бы подумать и в эту сторону (что в среднем человек за неделю 3 раза пользуется метро итд...), отследить по людям всякое, но это долго, и я не хочу
df.ticket_id.nunique()

335533

In [15]:
#работаем по стандарту поэтому, удаляем все повторяющеесе и не нужное
df = df.drop(columns=['id', 'ticket_id','entrance_id','entrance_nm','station_nm','line_id','line_nm'],axis = 1)
df

,ticket_type_nm,station_id,pass_dttm,time_to_under,label
0,Пропуск FacePay,11007,2022-09-12 05:00:13,216.316667,8001
1,СК учащегося 30 дней,2006,2022-09-12 05:00:54,648.183333,9011
2,БСК дружинника г.Москвы,2006,2022-09-12 05:00:55,865.333333,7022
3,30 дней,2006,2022-09-12 05:01:13,1048.233333,2022
4,КОШЕЛЕК,2006,2022-09-12 05:02:55,965.600000,2017
...,...,...,...,...,...
1091016,Пропуск FacePay,15005,2022-09-19 02:54:50,124.750000,15005
1091017,Пропуск FacePay,6007,2022-09-19 03:06:02,56.066667,6007
1091018,Пропуск FacePay,2010,2022-09-19 03:17:00,91.283333,2010
1091019,Пропуск FacePay,11014,2022-09-19 03:23:30,47.900000,11014


In [3]:
df['ticket_type_nm'].unique()

array(['Пропуск FacePay', 'СК учащегося 30 дней',
       'БСК дружинника г.Москвы', '30 дней', 'КОШЕЛЕК',
       'СК студента 30 дней', '90 дней', '90 дней ЕДИНЫЙ ТК',
       '30 дней СК учащегося', '90 дней СК студента',
       '30 дней СК студента', 'ББК', 'СК аспиранта 90 дней',
       'Социальная карта москвича', '30 дней ЕДИНЫЙ ТК',
       'Социальная карта жителя Моск. области', '30 дней Пригород ТК',
       'Социальная карта москвича с сопровождающим',
       'Пропуск сотрудника УВД по охране Метрополитена',
       '60 поездок ЕДИНЫЙ ТК', 'СК для сотрудника ГУВД г.Москвы',
       '365 дней ЕДИНЫЙ ТК', 'ВЕСБ МОСКВА (7мин)', 'СК студента 90 дней',
       '90 дней Пригород ТК', '30 дней  Пригород', '365 дней Пригород ТК',
       '365 дней', 'СК ординатора 30 дней', '30 дней СК аспиранта',
       '30 дней СК ординатора', '90 дней СК ординатора',
       'ВЕСБ МО (7 мин)', 'ВЕСБ МО (тип 5)', 'СК ординатора 90 дней',
       'СК аспиранта 30 дней', '90 дней СК учащегося',
       'Времен

In [16]:
# эти люди вынуждены ебашить на работу каждую неделю 5/2, иногда 6/1
# им выгодно тратить деньги раз в месяц или год ну или просто меньше, у студентов впринципе нет ничего, а работяги ведь работают не чтобы на транспорт тратиться
hardworkers = ['Пропуск FacePay','БСК дружинника г.Москвы','СК учащегося 30 дней','30 дней',
       'СК студента 30 дней', '90 дней', '90 дней ЕДИНЫЙ ТК',
       '30 дней СК учащегося', '90 дней СК студента',
       '30 дней СК студента', 'СК аспиранта 90 дней','30 дней ЕДИНЫЙ ТК','30 дней Пригород ТК',
       '60 поездок ЕДИНЫЙ ТК','365 дней ЕДИНЫЙ ТК', 'СК студента 90 дней',
       '90 дней Пригород ТК', '30 дней  Пригород', '365 дней Пригород ТК',
       '365 дней', 'СК ординатора 30 дней', '30 дней СК аспиранта',
       '30 дней СК ординатора', '90 дней СК ординатора','СК ординатора 90 дней',
       'СК аспиранта 30 дней', '90 дней СК учащегося',
       'СК учащегося 90 дней', '90 дней  Пригород',
       '90 дней СК аспиранта', '365 дней  Пригород',
       'СК ассистента-стажера 30 дней', 'СК ассистента-стажера 90 дней',
       '30 дней СК ассистента-стажера','Ультралайт Единый (70)','СК для сотрудника ГУВД г.Москвы']

# выделим их отдельно как почётных пользователей метро
# df.loc[df['ticket_type_nm'].isin(hardworkers), 'ticket_type_nm'] = 'трудяги-работяги'
df.loc[df['ticket_type_nm'].isin(hardworkers), 'ticket_type_nm'] = 1
# челы приехали почиллить из России в Москву, или из Китая, хз откуда еще приехать можно теперь, им билеты надолго не нужны
chillers = ['КОШЕЛЕК','ББК','Временный билет ММ','3 суток Единый ТК','Безналичная транспортная карта','1 сутки ЕДИНЫЙ ТК',
            '1 сутки Пригород ТК','Билет 3 суток ЕДИНЫЙ','Билет 1 сутки ЕДИНЫЙ']
# df.loc[df['ticket_type_nm'].isin(chillers), 'ticket_type_nm'] = 'чиллеры'
df.loc[df['ticket_type_nm'].isin(chillers), 'ticket_type_nm'] = 2
# просто челы с льготами, могут куда угодно вообще ехать и сколько угодно находиться в метро, вообще абстрактные в этом плане челы
lgotniki = ['Социальная карта москвича','Социальная карта жителя Моск. области',
       'Социальная карта москвича с сопровождающим',
       'ВЕСБ МОСКВА (7мин)',
       'ВЕСБ МО (7 мин)', 'ВЕСБ МО (тип 5)',
       'Временный билет ММ',
       'Социальная карта жителя Моск. области с сопровождающим',
       'ВЛБ МОСКВА', 'ВЕСБ МО с сопровождающим','ВЕСБ МО (тип 6)','ВЕСБ МОСКВА']
# df.loc[df['ticket_type_nm'].isin(lgotniki), 'ticket_type_nm'] = 'льготники'
df.loc[df['ticket_type_nm'].isin(lgotniki), 'ticket_type_nm'] = 3
# работяги метро работают в метро поэтому у них другие пропуска в метро и они хоть весь день там могут просидеть
metroworkers = ['Пропуск сотрудника УВД по охране Метрополитена',
       'СК для сотрудника ГУВД г.Москвы',
       'Пропуск сотрудника  метрополитена', 
       'Пропуск руководителя метрополитена', 'Пропуск  сотрудника МЦК']
# df.loc[df['ticket_type_nm'].isin(metroworkers), 'ticket_type_nm'] = 'работяги метро'
df.loc[df['ticket_type_nm'].isin(metroworkers), 'ticket_type_nm'] = 4
df['ticket_type_nm'].unique()


array([1, 2, 3, 4], dtype=object)

In [17]:
df['pass_dttm'] = pd.to_datetime(df['pass_dttm'])
df['hour'] = df.pass_dttm.apply(lambda x: x.hour)
df['day_of_week'] = df.pass_dttm.dt.weekday
df = df.drop(['pass_dttm'], axis=1)
df

,ticket_type_nm,station_id,time_to_under,label,hour,day_of_week
0,1,11007,216.316667,8001,5,0
1,1,2006,648.183333,9011,5,0
2,1,2006,865.333333,7022,5,0
3,1,2006,1048.233333,2022,5,0
4,2,2006,965.600000,2017,5,0
...,...,...,...,...,...,...
1091016,1,15005,124.750000,15005,2,0
1091017,1,6007,56.066667,6007,3,0
1091018,1,2010,91.283333,2010,3,0
1091019,1,11014,47.900000,11014,3,0


In [6]:
df.day_of_week.unique()

array([0, 1, 2, 3, 4, 5, 6])

In [7]:
df.hour.unique()

array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
       22, 23,  0,  1,  2,  3,  4])

In [8]:
# morning = [ 5,  6,  7,  8,  9]
# day = [10, 11, 12, 13, 14, 15, 16]
# evening = [17, 18, 19, 20, 21, 22]
# night = [23,  0,  1,  2,  3,  4]
# # df.loc[df['hour'].isin(morning), 'hour'] = 'утро'
# # df.loc[df['hour'].isin(day), 'hour'] = 'день'
# # df.loc[df['hour'].isin(evening), 'hour'] = 'вечер'
# # df.loc[df['hour'].isin(night), 'hour'] = 'ночь'
# df.loc[df['hour'].isin(morning), 'hour'] = 1
# df.loc[df['hour'].isin(day), 'hour'] = 2
# df.loc[df['hour'].isin(evening), 'hour'] = 3
# df.loc[df['hour'].isin(night), 'hour'] = 4
# weekends = [5,6]
# days = [0,1,2,3,4]
# df.loc[df['day_of_week'].isin(weekends), 'day_of_week'] = 0
# df.loc[df['day_of_week'].isin(days), 'day_of_week'] = 1

# df

,ticket_type_nm,station_id,time_to_under,label,hour,day_of_week
0,1,11007,216.316667,8001,4,1
1,1,2006,648.183333,9011,4,1
2,1,2006,865.333333,7022,4,1
3,1,2006,1048.233333,2022,4,1
4,2,2006,965.600000,2017,4,1
...,...,...,...,...,...,...
1091016,1,15005,124.750000,15005,4,1
1091017,1,6007,56.066667,6007,4,1
1091018,1,2010,91.283333,2010,4,1
1091019,1,11014,47.900000,11014,4,1


In [18]:
from sklearn.model_selection import train_test_split
X = df.drop(columns=['time_to_under', 'label'])
y_r = df['time_to_under']
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X, y_r, test_size=0.01)
y_c = df['label']
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X, y_c, test_size=0.01)

In [19]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

model = LinearRegression()
model.fit(X_train_r,y_train_r)
y_pred = model.predict(X_test_r)
r2 = r2_score(y_test_r,y_pred)
r2

0.4620677173278951

In [20]:
from sklearn.tree import DecisionTreeClassifier
logreg = DecisionTreeClassifier()
logreg.fit(X_train_c, y_train_c)
y_predict = logreg.predict(X_test_c)

In [21]:
from sklearn.metrics import recall_score
recall = recall_score(y_test_c, y_predict, average='micro')
print(recall)

0.052974062872330675


In [22]:
res = 0.5*r2+0.5*recall
res

0.25752089010011286

In [ ]:
# from sklearn.model_selection import train_test_split
# X = df.drop(columns=['time_to_under', 'label'])
# y = df['time_to_under']
# X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X, y, test_size=0.01)
# y = df['label']
# X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X, y, test_size=0.01)

In [ ]:
# !pip install catboost
# from catboost import CatBoostRegressor
# from catboost import CatBoostClassifier
# from sklearn.metrics import r2_score
# from sklearn.metrics import recall_score

# r_model = CatBoostRegressor(iterations=2,
#                             learning_rate=1,
#                             depth=2)
# c_model = CatBoostClassifier(iterations=2,
#                              learning_rate=1,
#                              depth=2)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# r_model.fit(X_train_r, y_train_r)
# time_to_under_predicted = r_model.predict(X_test_r)

0:	learn: 226.9574514	total: 164ms	remaining: 164ms
1:	learn: 226.7319940	total: 283ms	remaining: 0us


In [ ]:
# c_model.fit(X_train_c, y_train_c)
# labels_predicted = c_model.predict(X_test_c)

In [ ]:
# result = 0.5*r2_score(y_test_r, time_to_under_predicted) + 0.5*recall_score(y_test_c, labels_predicted, average='micro')
# result

NameError: ignored